In [40]:
from bs4 import BeautifulSoup
from IPython.core.display import display, HTML
import requests
import pandas as pd
import regex as re
import time

import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

import seaborn as sns

In [58]:
def pdp_scrape(url):
    response = requests.get(url)
    status = response.status_code
    time.sleep(1)

    if status == 200:
        soup = BeautifulSoup(response.text, 'html5lib')
        listing_end = str(soup.find('span', class_ = 'statusContent').text).find('This listing has ended')
        
        if listing_end < 0:
            name = soup.find('h1', id = 'itemTitle').text.split('\xa0')[1]
            condition = soup.find('div', itemprop='itemCondition').text
            try:
                sold_price = soup.find('span', class_ ='notranslate').text.split('$')[1].strip()
            except:
                sold_price = 0
            try:
                sold = soup.find('span', class_ = re.compile(r'vi-qty-pur-lnk$')).text.split('\t\t\t')[1].split(' ')[0]
            except:
                sold = 1
            year_sold = 2018
            try:
                free_shipping = 1 if str(soup.find('span', id = 'fshippingCost').text).find('FREE') > 0 else 0
            except:
                free_shipping = 1
            try:
                accept_return = 1 if str(soup.find('span', id = 'vi-ret-accrd-txt').text).find('not') < 0 else 0
            except:
                accept_return = 1
            result = [url, status, name, condition ,sold, year_sold, sold_price, free_shipping, accept_return]
        else:
            result = [url, 'listing_ended', 0, 0, 0, 0, 0, 0, 0]
    else:
        result = [url, status, 0, 0, 0, 0, 0, 0, 0]
    
    return result   

In [62]:
for page in range(15,0,-1):
    response = requests.get('https://www.ebay.com/sch/i.html?_fsrp=1&Network=Unlocked&_nkw=iphone&_sacat=0&_dmd=1&_from=R40&LH_BIN=1&LH_ItemCondition=3000&_dcat=9355&LH_Sold=1&Brand=Apple&_ipg=200&_pgn=' + str(page))
    soup = BeautifulSoup(response.text, 'html5lib')
    urls_raw = soup.find_all('a', class_ = 's-item__link')
    url_list = [ url.get('href') for url in urls_raw]
    
    print('Number of links found on Page ' + str(page) + ' : ' + str(len(url_list)))

    df = pd.DataFrame(columns=['URL', 'response', 'name', 'condition', 'sold', 'year_sold', 'sold_price', 'free_shipping', 'accept_return'])

    i = 0
    for url in url_list:
        print(str(page), str(i), url)
        i = i + 1
        result = pdp_scrape(url)
        df = df.append({column: result[i] for column, i in zip(df.columns,range(len(df.columns)))}, ignore_index=True)

    dfs = dfs.append(df, ignore_index=True)

Number of links found on Page 15 : 127
15 0 https://www.ebay.com/itm/Apple-iPhone-7-Plus-256GB-Silver-Unlocked-A1661-CDMA-GSM/202317724258?epid=225157592&hash=item2f1b137a62%3Ag%3ASD0AAOSwnHda5KZe&_fsrp=1&_pgn=15&_nkw=iphone&_sacat=0&_dmd=1&_from=R40&LH_BIN=1&LH_ItemCondition=3000&rt=nc&_dcat=9355&LH_Sold=1&_ipg=200
15 1 https://www.ebay.com/itm/Apple-iPhone-5C-16-GB-unlock-Pink/173318606235?epid=240131372&hash=item285a980d9b%3Ag%3AapcAAOSwbsta-gCp&_fsrp=1&_pgn=15&_nkw=iphone&_sacat=0&_dmd=1&_from=R40&LH_BIN=1&LH_ItemCondition=3000&rt=nc&_dcat=9355&LH_Sold=1&_ipg=200
15 2 https://www.ebay.com/itm/MINT-Apple-iPhone-8-Plus-Silver-64GB-Factory-Unlocked-Verizon-Warranty-READ/173324902622?epid=239054190&hash=item285af820de%3Ag%3AgOUAAOSwJdBa%7E0gV&_fsrp=1&_pgn=15&_nkw=iphone&_sacat=0&_dmd=1&_from=R40&LH_BIN=1&LH_ItemCondition=3000&rt=nc&_dcat=9355&LH_Sold=1&_ipg=200
15 3 https://www.ebay.com/itm/Factory-Unlocked-iPhone-6S-Plus-Gold-Silver-Space-Gray-ATT-Verizon-16-64GB-128GB/282100249169?ep

15 29 https://www.ebay.com/itm/Apple-iPhone-4s-16GB-Black-Unlocked-by-AT-T-A1387-CDMA-GSM-SIM-Free/302731599740?epid=177675960&hash=item467c359f7c%3Ag%3Ah4oAAOSwQTVa8NPP&_fsrp=1&_pgn=15&_nkw=iphone&_sacat=0&_dmd=1&_from=R40&LH_BIN=1&LH_ItemCondition=3000&rt=nc&_dcat=9355&LH_Sold=1&_ipg=200
15 30 https://www.ebay.com/itm/Apple-iPhone-4-32GB-White-AT-T-A1332-GSM-Great-Conditon/232757285808?epid=101872130&hash=item36316a53b0%3Ag%3AlFIAAOSwAata7N9v&_fsrp=1&_pgn=15&_nkw=iphone&_sacat=0&_dmd=1&_from=R40&LH_BIN=1&LH_ItemCondition=3000&rt=nc&_dcat=9355&LH_Sold=1&_ipg=200
15 31 https://www.ebay.com/itm/Apple-iPhone-8-64GB-Silver-Verizon-A1863-CDMA-GSM/253621256505?epid=239009482&hash=item3b0d014139%3Ag%3AAtQAAOSweNZa7IAT&_fsrp=1&_pgn=15&_nkw=iphone&_sacat=0&_dmd=1&_from=R40&LH_BIN=1&LH_ItemCondition=3000&rt=nc&_dcat=9355&LH_Sold=1&_ipg=200
15 32 https://www.ebay.com/itm/Apple-iPhone-4s-16GB-White-AT-T-A1387-Unlocked-MC924LL-A/123131870597?epid=1301497080&hash=item1cab3b3d85%3Ag%3ADtMAAOSwBY5a2P

15 58 https://www.ebay.com/itm/Apple-iPhone-5s-16GB-32GB-64GB-Gold-Gray-Silver-Factory-Unlocked-LTE/112773424326?epid=170038755&hash=item1a41d1e4c6%3Ag%3AFr0AAOSweLBaaJbp&var=413216779356&_fsrp=1&_pgn=15&_nkw=iphone&_sacat=0&_dmd=1&_from=R40&LH_BIN=1&LH_ItemCondition=3000&rt=nc&_dcat=9355&LH_Sold=1&_ipg=200
15 59 https://www.ebay.com/itm/Apple-iPhone-SE-128GB-Factory-Unlocked-iOS-12MP-Camera-Smartphone/362226837845?epid=240458785&hash=item545666e955%3Ag%3AIm8AAOSwA3dYD2P8&var=631440512193&_fsrp=1&_pgn=15&_nkw=iphone&_sacat=0&_dmd=1&_from=R40&LH_BIN=1&LH_ItemCondition=3000&rt=nc&_dcat=9355&LH_Sold=1&_ipg=200
15 60 https://www.ebay.com/itm/iPhone-4s-32GB-Used/112967908254?epid=109348360&hash=item1a4d697b9e%3Ag%3Axo0AAOSw0Pxa5i5s&_fsrp=1&_pgn=15&_nkw=iphone&_sacat=0&_dmd=1&_from=R40&LH_BIN=1&LH_ItemCondition=3000&rt=nc&_dcat=9355&LH_Sold=1&_ipg=200
15 61 https://www.ebay.com/itm/Apple-iPhone-SE-32GB-Gold-Unlocked-A1723-CDMA-GSM-BUNDLE-w-Case/173302557075?epid=240433994&hash=item2859a32993

15 88 https://www.ebay.com/itm/Apple-iPhone-8-Plus-256GB-Gold-Verizon-UNLOCKED-Virtually-Brand-New/273184007297?epid=239086429&hash=item3f9b092481%3Ag%3A53IAAOSwDPNa3y50&_fsrp=1&_pgn=15&_nkw=iphone&_sacat=0&_dmd=1&_from=R40&LH_BIN=1&LH_ItemCondition=3000&rt=nc&_dcat=9355&LH_Sold=1&_ipg=200
15 89 https://www.ebay.com/itm/Apple-iPhone-4S-Black-White-Factory-Unlocked-AT-T-T-Mobile-8-16GB-32GB-64GB/272220677391?epid=233930788&hash=item3f619de50f%3Ag%3ArQAAAOSw%7EoFXGnKy&var=571026606994&_fsrp=1&_pgn=15&_nkw=iphone&_sacat=0&_dmd=1&_from=R40&LH_BIN=1&LH_ItemCondition=3000&rt=nc&_dcat=9355&LH_Sold=1&_ipg=200
15 90 https://www.ebay.com/itm/Factory-Unlocked-iPhone-6-Plus-Gold-Silver-Space-Gray-ATT-Verizon-16-64GB-128GB/272212061190?epid=203711786&hash=item3f611a6c06%3Ag%3Aqm4AAOSw-YBXF9On&var=571017435665&_fsrp=1&_pgn=15&_nkw=iphone&_sacat=0&_dmd=1&_from=R40&LH_BIN=1&LH_ItemCondition=3000&rt=nc&_dcat=9355&LH_Sold=1&_ipg=200
15 91 https://www.ebay.com/itm/Apple-iPhone-6s-16GB-64GB-Gray-Gold-Silv

15 117 https://www.ebay.com/itm/Unlocked-Apple-iPhone-5-64GB-White-EUC/292410016626?epid=240131282&hash=item4414fec372%3Ag%3AvQMAAOSweZJaE71l&_fsrp=1&_pgn=15&_nkw=iphone&_sacat=0&_dmd=1&_from=R40&LH_BIN=1&LH_ItemCondition=3000&rt=nc&_dcat=9355&LH_Sold=1&_ipg=200
15 118 https://www.ebay.com/itm/Apple-iPhone-5S-32GB-Unlocked-GSM-Smartphone-Space-Gray/232683230400?epid=242594451&hash=item362d0054c0%3Ag%3AVJQAAOSw74FamByI&_fsrp=1&_pgn=15&_nkw=iphone&_sacat=0&_dmd=1&_from=R40&LH_BIN=1&LH_ItemCondition=3000&rt=nc&_dcat=9355&LH_Sold=1&_ipg=200
15 119 https://www.ebay.com/itm/Apple-iPhone-7-Plus-128GB-Silver-FACTORY-UNLOCKED-A1661-CDMA-GSM/332627557700?epid=241898532&hash=item4d7225bd44%3Ag%3AokIAAOSwPW9a2n3f&_fsrp=1&_pgn=15&_nkw=iphone&_sacat=0&_dmd=1&_from=R40&LH_BIN=1&LH_ItemCondition=3000&rt=nc&_dcat=9355&LH_Sold=1&_ipg=200
15 120 https://www.ebay.com/itm/Apple-iPhone-4S-32GB-White-AT-T-Smartphone-MC921LL-A/263621746845?epid=141200872&hash=item3d6114a09d%3Ag%3AEDUAAOSwJjZa2iq%7E&_fsrp=1&_p

...

1 65 https://www.ebay.com/itm/Apple-iPhone-5c-16GB-Blue-Unlocked-A1456-CDMA-GSM/163154797078?hash=item25fcc8a216%3Ag%3AV9QAAOSwQb5bTun%7E&_fsrp=1&_pgn=1&_nkw=iphone&_sacat=0&_dmd=1&_from=R40&LH_BIN=1&LH_ItemCondition=3000&rt=nc&_dcat=9355&LH_Sold=1&_ipg=200
1 66 https://www.ebay.com/itm/Apple-iPhone-5c-16GB-Yellow-Unlocked-A1456-CDMA-GSM/283062120571?hash=item41e7d1307b%3Ag%3AO20AAOSw2yNbTxci&_fsrp=1&_pgn=1&_nkw=iphone&_sacat=0&_dmd=1&_from=R40&LH_BIN=1&LH_ItemCondition=3000&rt=nc&_dcat=9355&LH_Sold=1&_ipg=200
1 67 https://www.ebay.com/itm/Apple-iPhone-7-128GB-Black-Verizon-Unlocked-A1660-CDMA-GSM/183332361259?hash=item2aaf75d42b%3Ag%3AWAgAAOSwkuVbT4fd&_fsrp=1&_pgn=1&_nkw=iphone&_sacat=0&_dmd=1&_from=R40&LH_BIN=1&LH_ItemCondition=3000&rt=nc&_dcat=9355&LH_Sold=1&_ipg=200
1 68 https://www.ebay.com/itm/Apple-iPhone-6-Plus-16GB-A1522-Space-Gray-UNLOCKED-item-346/123241381250?hash=item1cb1c23d82%3Ag%3A218AAOSw1qtbQ7X3&_fsrp=1&_pgn=1&_nkw=iphone&_sacat=0&_dmd=1&_from=R40&LH_BIN=1&LH_ItemCond

1 95 https://www.ebay.com/itm/Apple-iPhone-6s-64GB-Silver-UNLOCKED-CDMA-GSM/132708903057?hash=item1ee6112891%3Ag%3AN3MAAOSwF7lbT2mu&_fsrp=1&_pgn=1&_nkw=iphone&_sacat=0&_dmd=1&_from=R40&LH_BIN=1&LH_ItemCondition=3000&rt=nc&_dcat=9355&LH_Sold=1&_ipg=200
1 96 https://www.ebay.com/itm/Apple-iPhone-6-16GB-Space-Gray-Unlocked-EXCELLENT-COND/292646452793?epid=233752960&hash=item4423167e39%3Ag%3Aeo0AAOSwAzZbT6tB&_fsrp=1&_pgn=1&_nkw=iphone&_sacat=0&_dmd=1&_from=R40&LH_BIN=1&LH_ItemCondition=3000&rt=nc&_dcat=9355&LH_Sold=1&_ipg=200
1 97 https://www.ebay.com/itm/Apple-iPhone-7-Plus-128GB-Jet-Black-Unlocked-A1784-GSM/113157003706?epid=240307021&hash=item1a58aed9ba%3Ag%3Aoq4AAOSwNURbUBUw&_fsrp=1&_pgn=1&_nkw=iphone&_sacat=0&_dmd=1&_from=R40&LH_BIN=1&LH_ItemCondition=3000&rt=nc&_dcat=9355&LH_Sold=1&_ipg=200
1 98 https://www.ebay.com/itm/Apple-iPhone-7-128GB-Black-Verizon-A1660-CDMA-GSM-MNAJ2LL-A/192602572015?epid=225203245&hash=item2cd80204ef%3Ag%3ARVkAAOSwaENbT%7EZH&_fsrp=1&_pgn=1&_nkw=iphone&_sacat

1 125 https://www.ebay.com/itm/GSM-UNLOCKED-Apple-iPhone-6s-64GB-Space-Gray-AT-T-A1633-CDMA-GSM/223064232730?hash=item33efaa131a%3Ag%3ADrYAAOSwC6xbRB9I&_fsrp=1&_pgn=1&_nkw=iphone&_sacat=0&_dmd=1&_from=R40&LH_BIN=1&LH_ItemCondition=3000&rt=nc&_dcat=9355&LH_Sold=1&_ipg=200
1 126 https://www.ebay.com/itm/PREOWNED-USED-INBOX-Apple-iPhone-SE-16GB-GOLD-AT-T-GSM-UNLOCKED-A1662/162907257543?epid=236994051&hash=item25ee077ac7%3Ag%3AIb0AAOSwF7Faj0oK&_fsrp=1&_pgn=1&_nkw=iphone&_sacat=0&_dmd=1&_from=R40&LH_BIN=1&LH_ItemCondition=3000&rt=nc&_dcat=9355&LH_Sold=1&_ipg=200
1 127 https://www.ebay.com/itm/Apple-iPhone-6-Plus-16GB-Space-Gray-UNLOCKED/113156745662?hash=item1a58aae9be%3Ag%3An2YAAOSwUxhbT9nz&_fsrp=1&_pgn=1&_nkw=iphone&_sacat=0&_dmd=1&_from=R40&LH_BIN=1&LH_ItemCondition=3000&rt=nc&_dcat=9355&LH_Sold=1&_ipg=200
1 128 https://www.ebay.com/itm/Apple-iPhone-5S-16GB-Unlocked-Space-Gray-looks-very-good-works-great/292637623166?epid=168508202&hash=item44228fc37e%3Ag%3AglAAAOSwhmtbNR%7Ec&_fsrp=1&_pg

1 154 https://www.ebay.com/itm/Apple-iPhone-7-Plus-PRODUCT-RED-128GB-Unlocked-A1784-GSM/183332753553?epid=241178533&hash=item2aaf7bd091%3Ag%3Ast4AAOSwFYNbT2Bt&_fsrp=1&_pgn=1&_nkw=iphone&_sacat=0&_dmd=1&_from=R40&LH_BIN=1&LH_ItemCondition=3000&rt=nc&_dcat=9355&LH_Sold=1&_ipg=200
1 155 https://www.ebay.com/itm/Apple-iPhone-7-32GB-Rose-Gold-Unlocked-A1660-CDMA-GSM-Clean-ESN/183332486939?epid=225303158&hash=item2aaf77bf1b%3Ag%3AKLIAAOSwJ95bT5VG&_fsrp=1&_pgn=1&_nkw=iphone&_sacat=0&_dmd=1&_from=R40&LH_BIN=1&LH_ItemCondition=3000&rt=nc&_dcat=9355&LH_Sold=1&_ipg=200
1 156 https://www.ebay.com/itm/APPLE-IPHONE-3G-A1241-8GB-BLACK-FACTORY-UNLOCKED-MINT-CONDITION-RARE/223065451580?hash=item33efbcac3c%3Ag%3AfRIAAOSwuHJa70oQ&_fsrp=1&_pgn=1&_nkw=iphone&_sacat=0&_dmd=1&_from=R40&LH_BIN=1&LH_ItemCondition=3000&rt=nc&_dcat=9355&LH_Sold=1&_ipg=200
1 157 https://www.ebay.com/itm/Apple-iPhone-6-64GB-Unlocked-GSM-iOS-Smartphone/252178803513?epid=241111517&hash=item3ab7072b39%3Am%3AmKLfOWJnefIVcP3QoNCyeMw&va

1 183 https://www.ebay.com/itm/Apple-iPhone-7-Plus-PRODUCT-RED-128GB-Unlocked-A1661-CDMA-GSM/183318654533?epid=235221677&hash=item2aaea4ae45%3Ag%3A70UAAOSwX3FbEqAB&_fsrp=1&_pgn=1&_nkw=iphone&_sacat=0&_dmd=1&_from=R40&LH_BIN=1&LH_ItemCondition=3000&rt=nc&_dcat=9355&LH_Sold=1&_ipg=200
1 184 https://www.ebay.com/itm/Fair-Apple-iPhone-6-16GB-Space-Gray-Unlocked-Verizon-Smartphone-Free-Ship/153103928736?epid=240458865&hash=item23a5b48da0%3Ag%3A4AkAAOSw0hlZGfoj&_fsrp=1&_pgn=1&_nkw=iphone&_sacat=0&_dmd=1&_from=R40&LH_BIN=1&LH_ItemCondition=3000&rt=nc&_dcat=9355&LH_Sold=1&_ipg=200
1 185 https://www.ebay.com/itm/Apple-iPhone-6-16GB-Gold-Factory-Unlocked-GSM-4G-iOS-Smartphone/112995721930?epid=241025652&hash=item1a4f11e2ca%3Ag%3AQ6sAAOSw7aFa-u5V&_fsrp=1&_pgn=1&_nkw=iphone&_sacat=0&_dmd=1&_from=R40&LH_BIN=1&LH_ItemCondition=3000&rt=nc&_dcat=9355&LH_Sold=1&_ipg=200
1 186 https://www.ebay.com/itm/Apple-iPhone-6-16GB-Silver-A1586-CDMA-GSM-Unlocked/253760991931?hash=item3b155572bb%3Ag%3ATwoAAOSwScBbM

In [63]:
dfs.tail()

,URL,response,name,condition,sold,year_sold,sold_price,free_shipping,accept_return
3485,https://www.ebay.com/itm/In-OEM-Box-Apple-iPho...,200,In OEM Box Apple iPhone SE 32GB Space Gray (GS...,Used,4,2018,124.99,1,1
3486,https://www.ebay.com/itm/Apple-iPhone-8-Plus-2...,listing_ended,0,0,0,0,0,0,0
3487,https://www.ebay.com/itm/Apple-iPhone-6s-16GB-...,listing_ended,0,0,0,0,0,0,0
3488,https://www.ebay.com/itm/Near-Mint-Apple-iPhon...,200,Near Mint Apple iPhone 5C White 8GB Verizon Un...,Used,3,2018,63.00,1,1
3489,https://www.ebay.com/itm/Good-Apple-iPhone-5-B...,200,Good Apple iPhone 5 Black Color 16GB Factory U...,Used,15,2018,69.00,1,1


In [64]:
dfs4 = dfs
%store dfs4

Stored 'dfs4' (DataFrame)
